In [ ]:
'''
This was done in QuantConnect Platform (there are 3+ tutorials on universe selection), 
***WITHOUT PROPER MODULES IT WILL SHOW IMPORT ERRORS***
Personally, I prefer Quantopian / Alpaca, but those aren't as easy to trade live.
https://www.quantconnect.com/terminal/#bootcamp
'''
from datetime import timedelta
from QuantConnect.Data.UniverseSelection import * 
from Selection.FundamentalUniverseSelectionModel import FundamentalUniverseSelectionModel

class LiquidValueStocks(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2016, 10, 1)
        self.SetEndDate(2017, 10, 1)
        self.SetCash(100000)
        self.universe = None
        self.UniverseSettings.Resolution = Resolution.Hour
        self.AddUniverseSelection(LiquidValueUniverseSelectionModel())
        
        self.AddAlpha(LongShortEYAlphaModel()) #SPECIFY THIS FOR ENTRIES
        
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        self.SetExecution(ImmediateExecutionModel())
    
class LiquidValueUniverseSelectionModel(FundamentalUniverseSelectionModel):
    
    def __init__(self):
        self.lastMonth = -1 
        super().__init__(True, None, None)
    
    def SelectCoarse(self, algorithm, coarse):
        '''
        Selection based on Volume (sorted, takes top 100)
        and Fundamental Data present
        '''
        if self.lastMonth == algorithm.Time.month:
            return Universe.Unchanged
        self.lastMonth = algorithm.Time.month

        sortedByDollarVolume = sorted([x for x in coarse if x.HasFundamentalData],
            key=lambda x: x.DollarVolume, reverse=True)

        return [x.Symbol for x in sortedByDollarVolume[:100]]

    def SelectFine(self, algorithm, fine):
        '''
        Selection based on earnings yield (top and bottom 10 taken), 
        '''
        #1. Sort yields per share
        sortedByYields = sorted(fine, key=lambda f: f.ValuationRatios.EarningYield, reverse=True)
        
        #2. Take top 10 most profitable stocks -- and bottom 10 least profitable stocks
        self.universe = sortedByYields[:10] + sortedByYields[-10:]
        # Save to the variable self.universe

        #3. Return the symbol objects by iterating through self.universe with list comprehension
        return [u.Symbol for u in self.universe]
    
class LongShortEYAlphaModel(AlphaModel):

    def __init__(self):
        self.lastMonth = -1

    def Update(self, algorithm, data):
        insights = []
        
        #2. If else statement to emit signals once a month 
        if self.lastMonth == algorithm.Time.month:
            return insights
        self.lastMonth = algorithm.Time.month
        
        #3. For loop to emit insights with insight directions 
        # based on whether earnings yield is greater or less than zero once a month
        for security in algorithm.ActiveSecurities.Values:
            direction = 1 if security.Fundamentals.ValuationRatios.EarningYield > 0 else -1 
            insights.append(Insight.Price(security.Symbol, timedelta(28), direction)) 
        return insights
        # QC signals based on 'insights', which are rather confusing at first. 
        # Basically you append an Insight into a list of insights -- 
        # Insight can be based on many things, here it's based on price (Insight.Price())
        # Arguments to Insight.Price are symbol, lookback, and direction (1 for long, -1 for short) -- it's not my favorite.